In [1]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

Successfully connected to SQL database
success


In [2]:
%%sql
CREATE TABLE Employ (
    employee_id INTEGER,
    name TEXT,
    salary INTEGER
);

-- Sample Data
INSERT INTO Employ (employee_id, name, salary) VALUES
(1, 'Alice', 90000),
(2, 'Bob', 85000),
(3, 'Charlie', 90000),
(4, 'Diana', 80000),
(5, 'Eve', 70000);

 * sqlite:///question_bank.db
Done.
5 rows affected.


[]

In [3]:
%%sql
SELECT * FROM Employ

 * sqlite:///question_bank.db
Done.


employee_id,name,salary
1,Alice,90000
2,Bob,85000
3,Charlie,90000
4,Diana,80000
5,Eve,70000


In [4]:
%%sql
-- Rank the salaries and pick the second highest
-- RANK and DENSE RANK to rank salaries

SELECT salary, DENSE_RANK () OVER (ORDER BY salary DESC) AS rank
FROM Employ

-- DENSE_RANK() assigns the same rank to duplicates -- Handles Duplicates 
-- (it doesn't give different ranks to duplicates - keeps things fair enough)
-- We're ordering by salary descending, so:
-- Highest salary gets rank 1
-- Second highest gets rank 2
-- Then we filter for rank = 2.

 * sqlite:///question_bank.db
Done.


salary,rank
90000,1
90000,1
85000,2
80000,3
70000,4


In [5]:
%%sql

WITH Salrank AS (SELECT salary, DENSE_RANK () OVER (ORDER BY salary DESC) AS rank
                FROM Employ
            )
SELECT salary
FROM Salrank  -- name of the table that has salaries and thier ranks
WHERE rank =2;

 * sqlite:///question_bank.db
Done.


salary
85000


* WHY NOT ROW_NUMBER()
* Will not give same rank to duplicates
* This is correct only if all salaries are unique.

In [6]:
%%sql
SELECT salary, ROW_NUMBER () OVER (ORDER BY salary DESC) AS rank
FROM Employ

 * sqlite:///question_bank.db
Done.


salary,rank
90000,1
90000,2
85000,3
80000,4
70000,5


* Summary

1. ROW_NUMBER()	You want unique order (no ties)
2. RANK()	You want gaps in ranks for ties
3. DENSE_RANK()	You want ties without gaps

### Another scenario/method/different dataset

In [8]:
%%sql
-- Create the Employe table
CREATE TABLE Employe (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    salary INTEGER
);

-- Insert sample data
INSERT INTO Employe (employee_id, name, salary) VALUES
(1, 'Alice', 90000),
(2, 'Bob', 85000),
(3, 'Charlie', 90000),
(4, 'Diana', 80000),
(5, 'Eve', 70000),
(6, 'Frank', 80000);

 * sqlite:///question_bank.db
Done.
6 rows affected.


[]

In [9]:
%%sql
SELECT * FROM Employe

 * sqlite:///question_bank.db
Done.


employee_id,name,salary
1,Alice,90000
2,Bob,85000
3,Charlie,90000
4,Diana,80000
5,Eve,70000
6,Frank,80000


In [10]:
%%sql
SELECT salary, DENSE_RANK () OVER (ORDER BY salary DESC) AS ranked_salaries,employee_id
FROM Employe

 * sqlite:///question_bank.db
Done.


salary,ranked_salaries,employee_id
90000,1,1
90000,1,3
85000,2,2
80000,3,4
80000,3,6
70000,4,5


In [11]:
%%sql

WITH rh AS (SELECT salary, DENSE_RANK () OVER (ORDER BY salary DESC) AS ranked_salaries,employee_id
            FROM Employe
            )
SELECT e.name, e.salary, rh.ranked_salaries
FROM rh
JOIN Employe e
ON rh.employee_id=e.employee_id
WHERE ranked_salaries = 3
LIMIT 1;

 * sqlite:///question_bank.db
Done.


name,salary,ranked_salaries
Diana,80000,3
